In [4]:
import os, cv2, multiprocessing
import numpy as np

from concurrent.futures import ProcessPoolExecutor
from miseval import evaluate
from tqdm import tqdm

In [5]:
num_cores = multiprocessing.cpu_count()

In [6]:
label_daylight_img_root_path = 'C:\\workspace\\lane_result_img\\ground_truth\\daylight\\'
label_night_img_root_path = 'C:\\workspace\\lane_result_img\\ground_truth\\night\\'

pred_daylight_img_root_path = 'C:\\workspace\\lane_result_img\\predict\\\daylight\\'
pred_night_img_root_path = 'C:\\workspace\\lane_result_img\\predict\\night\\'

In [7]:
label_daylight_img_file_name_list = os.listdir(label_daylight_img_root_path)
label_daylight_img_file_name_list.sort()

label_night_img_file_name_list = os.listdir(label_night_img_root_path)
label_night_img_file_name_list.sort()

pred_daylight_img_file_name_list = os.listdir(pred_daylight_img_root_path)
pred_daylight_img_file_name_list.sort()

pred_night_img_file_name_list = os.listdir(pred_night_img_root_path)
pred_night_img_file_name_list.sort()

In [8]:
for i, img_file_name in enumerate(label_daylight_img_file_name_list):
    label_daylight_img_file_name_list[i] = label_daylight_img_root_path + img_file_name

for i, img_file_name in enumerate(label_night_img_file_name_list):
    label_night_img_file_name_list[i] = label_night_img_root_path + img_file_name
    
for i, img_file_name in enumerate(pred_daylight_img_file_name_list):
    pred_daylight_img_file_name_list[i] = pred_daylight_img_root_path + img_file_name

for i, img_file_name in enumerate(pred_night_img_file_name_list):
    pred_night_img_file_name_list[i] = pred_night_img_root_path + img_file_name

In [9]:
daylight_metric_val_arr = np.zeros((len(label_daylight_img_file_name_list), 10), dtype=np.float64)
night_metric_val_arr = np.zeros((len(label_night_img_file_name_list), 10), dtype=np.float64)

In [10]:
def calculate_metric(img_file_path):
    label_img = cv2.imread(filename=img_file_path[0], flags=cv2.IMREAD_GRAYSCALE)
    label_img = label_img >= 1
    label_img = label_img.astype(np.uint8)
    
    pred_img = cv2.imread(filename=img_file_path[1], flags=cv2.IMREAD_GRAYSCALE)
    pred_img = pred_img >= 1
    pred_img = pred_img.astype(np.uint8)
    
    acc_val = evaluate(label_img, pred_img, metric="ACC")
    prec_val = evaluate(label_img, pred_img, metric="PREC")
    dice_val = evaluate(label_img, pred_img, metric="DSC")
    iou_val = evaluate(label_img, pred_img, metric="IoU")
    hausdorff_val = evaluate(label_img, pred_img, metric="HD")
    tp_val = evaluate(label_img, pred_img, metric="TP")
    tn_val = evaluate(label_img, pred_img, metric="TN")
    fp_val = evaluate(label_img, pred_img, metric="FP")
    fn_val = evaluate(label_img, pred_img, metric="FN")
    auc_val = evaluate(label_img, pred_img, metric="AUC") 
    
    return acc_val, prec_val, dice_val, iou_val, hausdorff_val, tp_val, tn_val, fp_val, fn_val, auc_val

In [11]:
for i, img_path in enumerate(zip(tqdm(label_daylight_img_file_name_list), pred_daylight_img_file_name_list)):
    metric_values = calculate_metric(img_file_path=img_path)
    
    for j, metric in enumerate(metric_values):
        daylight_metric_val_arr[i, j] = metric

  0%|          | 121/60000 [00:05<41:50, 23.85it/s]


KeyboardInterrupt: 

In [9]:
split_label_daylight = np.array_split(label_daylight_img_file_name_list, num_cores)

for i, x in enumerate(split_label_daylight):
    split_label_daylight[i] = x.tolist()

In [10]:
split_pred_daylight = np.array_split(pred_daylight_img_file_name_list, num_cores)

for i, x in enumerate(split_pred_daylight):
    split_pred_daylight[i] = x.tolist()

In [ ]:
pool = ProcessPoolExecutor(max_workers=num_cores)
result = parmap.map(calculate_metric, split_label_daylight, pm_pbar=True, pm_processes=num_cores)

  0%|          | 0/20 [00:00<?, ?it/s]

In [33]:
print(np.mean(acc_val_night_arr))
print(np.mean(prec_val_night_arr))
print(np.mean(dice_val_night_arr))
print(np.mean(iou_val_night_arr))
print(np.mean(hausdorff_val_arr))

0.9779586340402279
0.7906375812949143
0.7724239959164715
0.6336110774351699
8.095732413719732
